In [24]:
!pip install transformers
!pip install torch
!pip install nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 890.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [27]:
stop_words = set(stopwords.words('english'))

def clean_tweet(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # remove URLs
    text = re.sub(r'\@\w+|\#','', text)  # remove @ and hashtags
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # remove punctuation
    tokens = word_tokenize(text)
    cleaned_tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    return ' '.join(cleaned_tokens)

df_sample['cleaned_text'] = df_sample['Tweets'].apply(clean_tweet)
df_sample[['Tweets', 'cleaned_text']].head()


,Tweets,cleaned_text
2899,Le #DessinDePresse de Sanaga : ls sont morts c...,dessindepresse sanaga sont morts comme ils ont...
594,#Russia #Wagner #RussiaCivilWar https://t.co/P...,russia wagner russiacivilwar
2870,Exclusive content -https://t.co/oEiSIIB2Z1\n.\...,exclusive content cosplay japan titan titanics...
52,Auch heute geht die politische Nachricht des T...,auch heute geht die politische nachricht des t...
1391,@crazyclipsonly Same type that would take a ho...,type would take homemade playstationcontrolled...


In [29]:
# Load pipeline
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [36]:
from transformers import AutoTokenizer, pipeline
from tqdm import tqdm

# Load sentiment pipeline and tokenizer
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_pipeline = pipeline("sentiment-analysis", model=model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

sentiments = []
scores = []

for text in tqdm(df_sample['cleaned_text']):
    # Use tokenizer to truncate properly to 512 tokens
    encoded_input = tokenizer(text, truncation=True, max_length=512, return_tensors="pt")

    # Get prediction from model
    result = sentiment_pipeline(tokenizer.decode(encoded_input['input_ids'][0], skip_special_tokens=True))[0]
    sentiments.append(result['label'])
    scores.append(result['score'])

# Add to dataframe
df_sample['transformer_sentiment'] = sentiments
df_sample['transformer_score'] = scores


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu
100%|██████████| 500/500 [00:55<00:00,  9.05it/s]


In [38]:
df_sample[['Tweets', 'truncated_text', 'transformer_sentiment', 'transformer_score']].head(10)


,Tweets,truncated_text,transformer_sentiment,transformer_score
2899,Le #DessinDePresse de Sanaga : ls sont morts c...,dessindepresse sanaga sont morts comme ils ont...,NEGATIVE,0.989421
594,#Russia #Wagner #RussiaCivilWar https://t.co/P...,russia wagner russiacivilwar,NEGATIVE,0.962062
2870,Exclusive content -https://t.co/oEiSIIB2Z1\n.\...,exclusive content cosplay japan titan titanics...,NEGATIVE,0.961531
52,Auch heute geht die politische Nachricht des T...,auch heute geht die politische nachricht des t...,NEGATIVE,0.979176
1391,@crazyclipsonly Same type that would take a ho...,type would take homemade playstationcontrolled...,NEGATIVE,0.994880
807,#SUGA_AgustD_TOUR_in_Seoul #SUGA_AgustD_TOUR #...,sugaagustdtourinseoul sugaagustdtour glastonbu...,NEGATIVE,0.970569
2761,#Titan mishap: #Implosion with incredible forc...,titan mishap implosion incredible force amp sp...,NEGATIVE,0.989431
196,#Wagner #Russia,wagner russia,POSITIVE,0.992612
1576,#merri le #titanic 2 le retour https://t.co/4s...,merri titanic retour via,POSITIVE,0.505951
670,"Il Segretario di Stato americano #Blinken: ""no...",segretario stato americano blinken non credo c...,NEGATIVE,0.933375
